# 🚀 Semantic Kernel Demo Notebook

Welcome to the **Semantic Kernel Demo Notebook**! 🎉

In this notebook, we'll explore the powerful capabilities of the Semantic Kernel, a cutting-edge tool designed to enhance your data processing and analysis workflows. Whether you're a data scientist, developer, or just curious about semantic technologies, this demo will provide you with a hands-on experience of how the Semantic Kernel can transform your projects.

## What You'll Learn 📚

- **Introduction to Semantic Kernel**: Understand the core concepts and architecture.
- **Data Processing**: See how the Semantic Kernel can streamline your data workflows.
- **Real-World Applications**: Discover practical use cases and examples.

## Let's Get Started! 🏁

Ready to dive in? Follow along with the cells below and unleash the power of the Semantic Kernel in your projects. Happy coding! 💻

### Import Required Libraries

In [ ]:
%pip install semantic-kernel==1.17.1
%pip install python-dotenv==1.0.1

### Load the necessary environments variables

In [2]:
import os
from dotenv import load_dotenv

#  If we change the .env file you want to override the values loaded in memory
load_dotenv(override=True)

chat_deployment_name = os.getenv("CHAT_DEPLOYMENT_NAME")
api_key = os.getenv("OPENAI_API_KEY")
base_url = os.getenv("OPENAI_BASE_URL")

### Let's use the Semantic Kernel following those steps: 🤖✨

1. **Select the Best AI Service 🏆**
   - Choose the most suitable AI service to run your prompt.

2. **Build the Prompt 🛠️**
   - Construct the prompt using the provided prompt template.

3. **Send the Prompt 🚀**
   - Dispatch the prompt to the selected AI service.

4. **Receive and Parse the Response 📬**
   - Collect and interpret the response from the AI service.

5. **Return the Response 🔄**
   - Deliver the response from the LLM back to your application.



In [9]:
import asyncio
import logging

from semantic_kernel import Kernel
from semantic_kernel.utils.logging import setup_logging
from semantic_kernel.functions import kernel_function
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.chat_completion_client_base import ChatCompletionClientBase
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.functions.kernel_arguments import KernelArguments

from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureChatPromptExecutionSettings,
)

# Creating the Kernel object

kernel =Kernel()

chat_completion = AzureChatCompletion(
        service_id="azure_openai_chat_completion",
        deployment_name=chat_deployment_name,
        api_key=api_key,
        endpoint=base_url
)

# Adding the chat completion service to the kernel
kernel.add_service(chat_completion)

# Set the logging level of the kernel to DEBUG
setup_logging()
logging.basicConfig(
    format="[%(asctime)s - %(name)s:%(lineno)d - %(levelname)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)
logging.getLogger("kernel").setLevel(logging.DEBUG)


In [ ]:
kernel.services.keys()

In [ ]:
kernel.get_service("azure_openai_chat_completion")

In [10]:
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings

execution_settings = OpenAIChatPromptExecutionSettings(
    max_tokens=1000,
    temperature=0.7,
)

chat_completion_service = kernel.get_service(service_id="azure_openai_chat_completion")

print(chat_completion_service)

ai_model_id='gpt-4o' service_id='azure_openai_chat_completion' client=<openai.lib.azure.AsyncAzureOpenAI object at 0x7fd6746c1a00> ai_model_type=<OpenAIModelTypes.CHAT: 'chat'> prompt_tokens=0 completion_tokens=0 total_tokens=0


In [11]:
chat_history = ChatHistory()

chat_history.add_user_message("What is the best dishes in all Quebec ?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

In [ ]:
print(response)

In [13]:
chat_history.add_assistant_message(response.content)

In [ ]:
print(chat_history)

In [18]:
# Save all chat history to a file

chat_history.store_chat_history_to_file("chat_history.json")

In [16]:
chat_history.add_user_message("Give me the recipe how to do the first dish you mentioned ?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

In [ ]:
print(response.content)

In [ ]:
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.contents import ChatMessageContent

chat_history = ChatHistory()

chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.SYSTEM,
        content="""You are an AI assistant, you provide food recipes only from Saguenay Lac-Saint-Jean.  
                 If someone asks you for a recipe from another region, you will say that you don't know it and give 
                 an alternative from Saguenay Lac-St-Jean.""",
    )
)

chat_history.add_message(
    ChatMessageContent(
        role=AuthorRole.USER,
        content="What is the 10 best dishes you have ?",
    )
)

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

chat_history.add_assistant_message(response.content)

print(response.content)

In [ ]:
chat_history.add_user_message("Give me the best recipe from Texas USA ?")

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings,
)

chat_history.add_assistant_message(response.content)

print(response.content)

You can provide image to get some information about it


In [ ]:
from semantic_kernel.contents import ChatHistory, ChatMessageContent, ImageContent, TextContent

chat_history = ChatHistory()

with open("link.txt", "r") as file:
    file_path = file.readline().strip()

chat_history.add_message(
    ChatMessageContent(
        role="user",
        items=[
            TextContent(text="What’s in this image?"),
            ImageContent(uri=file_path),
        ]
    )
)

response = await chat_completion.get_chat_message_content(
    chat_history=chat_history,
    settings=execution_settings
)
print(response)

---

# 🚀 Chat Completion with Function Calling

The most powerful feature of chat completion is the ability to call functions from the model. This enables you to:

- 🤖 Create chat bots that interact with your existing code
- 🔄 Automate business processes
- 💻 Generate code snippets

## 🌟 Simplified with Semantic Kernel

Semantic Kernel simplifies this process by:

- 📜 Automatically describing your functions and their parameters to the model
- 🔄 Handling the communication between the model and your code

## 🧠 Understanding the Process

To optimize your code and fully leverage this feature, it's important to understand the behind-the-scenes workings of function calling.

---

Create the service needed for the plugin

In [32]:
# Inline function
from semantic_kernel.prompt_template import InputVariable, PromptTemplateConfig

prompt = """
   Tell me a joke based on this characters {{$input}} from
   the nintendo universe.  If this characters is not from the nintendo universe,
   just said that you don't know joke outside the Nintendo universe.
"""

execution_settings = AzureChatPromptExecutionSettings(
    service_id="azure_openai_chat_completion",
    ai_model_id="gpt-35-turbo",
    max_tokens=2000,
    temperature=0.7,
)

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="nintendoJoke",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="input", description="The user input", is_required=True),
    ],
    execution_settings=execution_settings,
)

joke = kernel.add_function(
    function_name="nitendoJokeFunc",
    plugin_name="jokePlugin",
    prompt_template_config=prompt_template_config,
)


In [ ]:
input = "Mario and Luigi"

funnyJoke = await kernel.invoke(joke, input=input)

print(funnyJoke)

In [ ]:
input = "Sonic the hedgehog"

funnyJoke = await kernel.invoke(joke, input=input)

print(funnyJoke)

# Create a plugin for the Semantic Kernel

In [45]:
from semantic_kernel.functions import kernel_function

class BookMeetingRoomPlugin:
    def __init__(self):
        self.conf_rooms = [{
            "name": "Room Azure View",
            "capacity": 10,
            "available": True
        }, {
            "name": "Room Office View",
            "capacity": 20,
            "available": True
        }, {
            "name": "Room Github View",
            "capacity": 30,
            "available": True
        }]
    
    @kernel_function(description="Get list of booking rooms available")
    def get_list_of_rooms_available(self):
        return [room["name"] for room in self.conf_rooms if room["available"]]

    @kernel_function(description="Book a meeting room and return the name of the available room")
    def book_room(self,number_of_people: int):
        for room in self.conf_rooms:
            if room["available"] and room["capacity"] >= number_of_people:
                room["available"] = False
                return f"Room {room['name']} has been booked for {number_of_people} people."
        return "No available rooms found."
    
    @kernel_function(description="Cancel a meeting room and return the cancellation status")
    def cancel_room(self, room_name: str):
        for room in self.conf_rooms:
            if room["name"] == room_name:
                room["available"] = True
                return f"Room {room['name']} has been cancelled."
        return "Room not found."

In [46]:
# Register the plugin
kernel.remove_all_services()


kernel.add_plugin(BookMeetingRoomPlugin(),"meetingRoomPlugin")

KernelPlugin(name='meetingRoomPlugin', description=None, functions={'book_room': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='book_room', plugin_name='meetingRoomPlugin', description='Book a meeting room and return the name of the available room', parameters=[KernelParameterMetadata(name='number_of_people', description=None, default_value=None, type_='int', is_required=True, type_object=<class 'int'>, schema_data={'type': 'integer'}, include_in_function_choices=True)], is_prompt=False, is_asynchronous=False, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='Any', is_required=True, type_object=None, schema_data={'type': 'object'}, include_in_function_choices=True), additional_properties={}), invocation_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x7fd6725b53d0>, streaming_duration_histogram=<opentelemetry.metrics._internal.instrument._ProxyHistogram object at 0x7fd6725b5910>, 

In [53]:
# Booking a room

#chat_completion = kernel.get_service(type=ChatCompletionClientBase)

execution_settings = AzureChatPromptExecutionSettings()
execution_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

chat_history = ChatHistory()

chat_history.add_user_message("I will like to have the available booking rooms please")

response = (await chat_completion.get_chat_message_contents(
      chat_history=chat_history,
      settings=execution_settings,
      kernel=kernel,
      arguments=KernelArguments(),
  ))[0]

chat_history.add_assistant_message(response.content)

print(response.content)


The available booking rooms are:

1. **Room Azure View**
2. **Room Office View**
3. **Room Github View**


In [ ]:
chat_history.add_user_message("Which room is available for 20 people and if possible book it?")

response = (await chat_completion.get_chat_message_contents(
      chat_history=chat_history,
      settings=execution_settings,
      kernel=kernel,
      arguments=KernelArguments(),
  ))[0]

chat_history.add_assistant_message(response.content)

print(response.content)

The room **Room Office View** has been successfully booked for 20 people.


In [55]:
chat_history.add_user_message("I will like to have the available booking rooms please")

response = (await chat_completion.get_chat_message_contents(
      chat_history=chat_history,
      settings=execution_settings,
      kernel=kernel,
      arguments=KernelArguments(),
  ))[0]

chat_history.add_assistant_message(response.content)

print(response.content)

The currently available booking rooms are:

1. **Room Azure View**
2. **Room Github View**


In [51]:
chat_history.add_user_message("Can you cancel my booking ?")

response = (await chat_completion.get_chat_message_contents(
      chat_history=chat_history,
      settings=execution_settings,
      kernel=kernel,
      arguments=KernelArguments(),
  ))[0]

chat_history.add_assistant_message(response.content)

print(response.content)

Your booking for Room Office View has been successfully canceled.


In [52]:
chat_history.add_user_message("I will like to have the available booking rooms please")

response = (await chat_completion.get_chat_message_contents(
      chat_history=chat_history,
      settings=execution_settings,
      kernel=kernel,
      arguments=KernelArguments(),
  ))[0]

chat_history.add_assistant_message(response.content)

print(response.content)

The currently available booking rooms are:

1. Room Azure View  
2. Room Office View  
3. Room Github View
